# Корпус: форматирование текстов и унификация формата

Тексты либретто собраны из разных источников и имеют разную природу (официально изданный текст, коллективная расшифровка аудио, рабочий файл с читки в театре, тексты из мобильного приложения, файлы от авторов и т.д. и т.п.)

Как следствие, они крайне неоднородны. Различаются:
* формат файла (txt, docx, doc, pdf)
* наличие / отсутствие технической информации: заметки о сценографии, визуальных/звуковых эффектах, деталях игры и т.д.
* оформление пометок о том, какие герои произносят текст (или их полное отсутствие)
* оформление названий номеров
* нумерация актов спектакля
* оформление информации о повторах строк в песнях

Либретто, которые парсились из интернета, изначально записывались в txt в нужном виде, однако там могут быть пометки, которые также не нужны нам.

Первоначальная задача - **унифицировать тексты** либретто, чтобы все они были отформатированы по единой логике - и далее их можно будет предобрабатывать и готовить к непосредственно работе с ними


In [2]:
import re
import os


## Базовые функции для редактирования текстов

Итак, что нам нужно найти в тексте, чтобы отформатировать по единому принципу, если оно там есть, а затем на следующем этапе работы единовременно удалить (сейчас эта информация не нужна, но в теории может при каких-то других задачах понадобится):
- *Пометки о частях спектакля*: 
Акт 1, Сцена 1 и т.д.

- *Названия песен*: 
ОЧЕНЬ ХОРОШАЯ ПЕСНЯ, 2 песня о любви (к Питону) и т.д.

- *Любой текст, который не звучит со сцены*: 
(гуляет по саду),  ( Падает решетка Гаснет свет), В студию входит ОКСАНА и т.д.

- *Указания на то, кто говорит/поет текст*:  
Смердяков -, Solo T & B:, ВМЕСТЕ:, хор, Пётр(в детстве): и т.д.

Говорящих/поющих указывают большим количеством способов оформления, поэтому с ними лучше работать отдельно. А остальное можно объединить в один шаг

In [54]:
def v2_actssongs_formatted(libretto):
    ''' Находит и форматирует пометки о начале актов и музыкальные номера по нужному нам шаблону: 
        - акты: "\n\n\n\nАКТ 1\n\n\n\n"
        - песни: "\n\n\n12. НАЗВАНИЕ\n\n" 
        - информация о сценографии, если есть: \n\n///Какой-то текст на сколько угодно строк///\n\n
        - обозначения сцен: \n\nСцена 1. НАЗВАНИЕ\n\n'''
    text = libretto
    # flags = re.DOTALL | re.MULTILINE 
    # DOTALL - точка означает абсолютно любой символ, включая переносы строк; 
    # MULTILINE - ^ и $ можно использовать по строкам, а не по документу

    text = re.sub(r'(?: )+\n', '\n', text) # убираем слипшиеся пробелы с переносами строки и заменяем на переносы
    text = re.sub(r'\n+', '\n', text) # убираем все многократные переносы строк, чтобы записать все так, как нам надо
    text = re.sub(r'(?: )+', ' ', text) # удаляем все повторяющиеся пробелы
    text = re.sub(r'^(?: )+', '', text, flags = re.MULTILINE) # удаляем пробелы в начале строк
    text = re.sub(r'\t+', '', text) # удаляем табуляцию

    set_info = re.findall(r'(?:\n*?)?\/{3}.+?\/{3}(?:\n*?)?', text, flags = re.DOTALL) # инфо о сценографии. не всегда есть разрыв строки перед/после
    # flags - чтобы взял абсолютно все символы между /// и ///, т.к. бывает, что внутри есть переносы строк
    if set_info:
        for i in set_info:
            text = re.sub(re.escape(i), '\n\n' + i.strip() + '\n\n', text)
    
    # убираем визуальный мусор
    if '/(' in text: 
            text = re.sub(re.escape('/('), '/', text)
    if ')/' in text:
            text = re.sub(re.escape(')/'), '/', text)

    songs = re.findall(r'\n+\d{1,2}[\.\)].+?\n+', text) # названия песен
    if songs:
        for i in songs:
            text = re.sub(re.escape(i), '\n\n\n' + i.strip().upper() + '\n\n', text)

    acts = re.findall(r'(?:\s*?)\n[Аа][Кк][Тт] [12I]I?\n(?:\s+)?', text) # акты. могут быть с римскими или арабскими цифрами
    if acts:
        for i in acts:
            text = re.sub(i, '\n\n\n\n' + i.strip().upper() + '\n\n\n\n', text)
    
    scenes = re.findall(r'\n+[Сцена|СЦЕНА|Картина|КАРТИНА] \d{1,2}\.?.*?\n+', text)
    if scenes:
        for i in scenes:
            text = re.sub(re.escape(i), '\n\n' + i.strip().upper() + '\n\n', text)

    
    return text

In [5]:
def v2_speakers_formatted(libretto):
    ''' Находит в текстах песен отметки о том, какие персонажи поют тот или иной фрагмент,
        и форматируют эту строку по нужному нам шаблону - "\n\nПЕРСОНАЖ:\nТекст песни..." '''
    text = libretto
    #speakers = set(re.findall(r'\n+[А-ЯЁа-яё -‒–—― \(\)\\\/]+(?: )?:(?: )?\n+', text)) #захватываем с переносом строк, чтобы привести к нужному нам формату
    speakers = set(re.findall(r'\n+[А-ЯЁа-яё -‒–—― \(\)\\\/]+(?: )?:(?: )*?\n+', text))
    
    # сортируем по длине в порядке убывания, чтобы не было ситуации, когда в строке типа "Персонаж1 и Персонаж2:" 
    # регулярка поймала только "Персонаж2:", который как отдельный элемент тоже есть, и получится "Персонаж1 и ПЕРСОНАЖ2:" (нам нужно капсом всё)
    speakers_fine = sorted(speakers, key=len, reverse = True)
    
    for i in speakers_fine:
       text = re.sub(re.escape(i), '\n\n' + i.strip().upper() + '\n', text)

    return text

### Файлы txt

In [ ]:
def txt_extraction(filename):
    '''Вытаскивает текст из txt-файла'''
    try: 
        with open(f'C:/Users/User/Desktop/КЛ/либретто/Берем/процесс редактирования/{filename}', 'r') as file:
            extracted_text = file.read()
    except UnicodeDecodeError:
        with open(f'C:/Users/User/Desktop/КЛ/либретто/Берем/процесс редактирования/{filename}', 'r', encoding = 'utf-8') as file:
            extracted_text = file.read()
    
    return extracted_text

### Файлы docx / doc

In [9]:
# %pip install docx2python
from docx2python import docx2python


In [10]:
def v2_docx_extraction(filename):
    '''Вытаскивает текст из docx-, doc-файлов
    (!) Работает не очень быстро '''
    with docx2python(f'C:/Users/User/Desktop/КЛ/либретто/Берем/процесс редактирования/{filename}') as doc:
        extracted_text = doc.text

    return extracted_text        

### Файлы PDF

In [11]:
# %pip install pdfminer.six
from pdfminer.high_level import extract_text

In [12]:
def pdf_extraction(filename):
    '''Вытаскивает текст из pdf-файла'''
    extracted_text = extract_text(f"C:/Users/User/Desktop/КЛ/либретто/Берем/процесс редактирования/{filename}")
    
    return extracted_text

In [13]:
# отредактируем прежнюю функцию для обработки txt - оставим только запись из файла в переменную, остальное будет делать большая функция
def txt_extraction(filename):
    '''Вытаскивает текст из txt-файла'''
    try: 
        with open(f'C:/Users/User/Desktop/КЛ/либретто/Берем/процесс редактирования/{filename}', 'r') as file:
            extracted_text = file.read()
    except UnicodeDecodeError:
        with open(f'C:/Users/User/Desktop/КЛ/либретто/Берем/процесс редактирования/{filename}', 'r', encoding = 'utf-8') as file:
            extracted_text = file.read()
    
    return extracted_text

## Универсальная функция для всех форматов текстов

### Версия для самых частых случаев

In [14]:
# Объединяем редактирование ВСЕХ возможных форматов файлов
def complete_formatting(filename):
    '''Единая функция для форматирования текстов в формате text, docx/doc и pdf.'''

    # вытаскиваем текст
    if filename.endswith('.txt'):
        extracted = txt_extraction(filename)
    elif filename.endswith('.pdf'):
        extracted = pdf_extraction(filename)
    elif filename.endswith(('.doc', '.docx')): # на входе - кортеж с разными вариантами
        extracted = docx_extraction(filename)
    else:
        print('Ошибка! Проверьте формат файла')
        pass

    # форматирование
    edited = v2_actssongs_formatted(extracted) # номера актов, названия песен и инфо о сценографии
    edited = v2_speakers_formatted(edited) # пометки о том, кто говорит/поет текст   
    
    # записываем отредактированный текст в файл txt
    # сначала задаем имя для файла в зависимости от типа
    if filename.endswith(('.pdf', '.doc', '.txt')):
        new_name = f'{filename[:-4]}_fin.txt'
    elif filename.endswith('.docx'):
        new_name = f'{filename[:-5]}_fin.txt'
    else:
        new_name = f'{filename[:-5]}_fin.txt' # мало ли, что бывает, но есл что-то не так - лучше отрезать побольше

    # сохранение отредактированного текста на компьютер
    with open(f'C:/Users/User/Desktop/КЛ/либретто/Берем/процесс редактирования{new_name}', 'w', encoding = 'utf-8') as file:
        file.write(edited)    

    return edited
    

### Универсальная функция с возможностью кастомизации

Поскольку все файлы очень разные, часто нужна более тонкая настройка. 
Для этого сначала нужно адаптировать базовые функции под прием паттернов

In [51]:
def custom_speakers_formatted(libretto, pattern):
    ''' Находит в текстах песен отметки о том, какие персонажи поют тот или иной фрагмент,
        и форматируют эту строку по нужному нам шаблону - "\n\nПЕРСОНАЖ:\nТекст песни..." 
        NB! Для менее распространенных случаев, которые не укладываются в наш базовый паттерн поиска - но мы можем выделить другой паттерн '''
    text = libretto
    speakers = set(re.findall(pattern, text)) # тело паттерна задается на входе
    
    # сортируем по длине в порядке убывания, чтобы не было ситуации, когда в строке типа "Персонаж1 и Персонаж2:" 
    # регулярка поймает только "Персонаж2:", который как отдельный элемент тоже есть, и получится "Персонаж1 и ПЕРСОНАЖ2:" (нам нужно капсом всё)
    speakers_fine = sorted(speakers, key=len, reverse = True)

    for i in speakers_fine:
       replacement = i.strip().upper() + ':'
       text = re.sub(re.escape(i), '\n\n' + replacement + '\n', text)

    # если в паттерне было двоеточие, то оно задваивается после редактирование - убираем повтор (функция в основном для случаев, 
    # когда двоеточия как раз нет, но иногда приходится прибегать к ней и для случаев, когда оно есть)
    if '::' in text: 
        text = re.sub(r'\:\:', ':', text)

    return text

In [ ]:
def custom_set_info(text, pattern):
    '''Форматирует информацию о сценографии, вычисляя ее по пользовательскому паттерну'''
    set_info = re.findall(pattern, text, flags = re.DOTALL) # инфо о сценографии. не всегда есть разрыв строки перед/после
    # flags - чтобы взял абсолютно все символы между /// и ///, т.к. бывает, что внутри есть переносы строк
    if set_info:
        for i in set_info:
            formatted = '///' + i.strip() + '///'
            text = re.sub(re.escape(i), '\n\n' + formatted + '\n\n', text)
        
        # если инфо о сценографии в скобках и мы идем от этого, то удалим их, чтобы они не мешались с нашей разметкой
        if '(' in pattern and ')' in pattern: 
            text = re.sub(re.escape('/('), '/', text)
            text = re.sub(re.escape(')/'), '/', text)
        
    else:
        print('Паттерн не сработал, попробуйте еще раз')

    return text

А также инструмент для удаления всякого мусора, который может оказаться в тексте, особенно если это не txt:

In [3]:
def special_removal(libretto, pattern):
    ''' Находит и удаляет конкретные символы/последовательности символов в тексте по запросу пользователя'''

    text = libretto
    text = re.sub(fr'{pattern}', '', text)

    return(text)

### Функция для форматирования либретто с настройкой параметров
Это позволит добиться порядка в тексте сразу за один шаг. Настраиваются:
- паттерн для определения не-разговорного текста, который нам в дальнейшем не будет нужен и его надо подготовить к удалению
- пометки о говорящих/поющих - возможность ввести свой паттерн и учитесть случаи, когда они в принципе не указаны (тогда нет смысл запускать эту часть кода, ведь мы рискуем зацепить лишние строки просто так, а не в качестве издержки форматирования этих пометок)
- специфические символы / наборы символов / фрагменты текста, которые не несут в себе никакой информации (или даже вредят содержанию - например, колонтитулы, номера страниц) и могут быть удалены

Суть функции:
1. Извлечение текста из файла подходящим способом в зависимости от его формата
2. Обработка и форматирование текста с коррекцией общего пути обработки в зависимости от указанных параметров
3. Беглая очистка от мусора, который мог образоваться в процессе
4. Сохранение готового текста на компьютер под новым названием

In [ ]:

def v2_complete_formatting_woptions(filename, set_info, speakers, to_remove):
    '''Единая функция для форматирования текстов в формате text, docx/doc и pdf для частных случаев. С возможностью ввода специфических паттернов,
        которые позволят обработать конкретно этот текст.
        Настройка параметров: 
        1. set_info - информация о сценографии: о действиях персонажей, технических моментах, характеристиках реплик типа "с подозрением", "тихо смеется" и т.д.
            - 'none' - явного паттерна нет или все стандартно по нашей схеме
            - ...any custom value here... - паттерн, если есть единый стиль, которым обозначена эта информация
        2. speakers - говорящие/поющие:
           - 'none' - все в порядке, доп. действия здесь не требуется
           - 'absent' - если они не указаны в тексте вообще (поможет избежать ненужных исправлений)
           - ...any custom value here... - паттерн, если у спикеров есть явный паттерн, но не тот, что мы взяли по умолчанию
        3. to_remove - специфические пометки, которые захламляют текст и не нужны (например, номера страниц и колонтитулы в пдф, примечания в тексте)
            - 'none' - пометки для удаления отсутствуют
            - ...any custom value here... - паттерн(ы) для удаления, задаются в виде списка

    '''

    # вытаскиваем текст
    if filename.endswith('.txt'):
        extracted = txt_extraction(filename)
    elif filename.endswith('.pdf'):
        extracted = pdf_extraction(filename)
    elif filename.endswith(('.doc', '.docx')): # на входе - кортеж с разными вариантами
        extracted = v2_docx_extraction(filename)
    else:
        print('Ошибка! Проверьте формат файла')
        pass

    edited = extracted

    # удаление специфических пометок:    
    if to_remove == 'none':
        edited = edited
    else:
        patterns_removal = to_remove
        print(f'Паттерн(ы) для спец. пометок: {patterns_removal}')

        for p in patterns_removal:
            edited = special_removal(edited, p)

    # акты, песни:
    edited = v2_actssongs_formatted(edited) # номера актов, названия песен и инфо о сценографии
    # вообще, там ^ зашито удаление инфо о сценографии по принятому за базовый паттерн (метка - '///'), но переписывать функцию и менять все очень лень
    # код все равно ничего не найдет и не поменяет, если там нет нашей разметки через '///'

    # инфо о сценографии:
    if set_info == 'none':
        edited = edited
    else:
        set_pattern = set_info
        print(f'Паттерн для сценографии: {set_pattern}')
        edited = custom_set_info(edited, set_pattern)

    # пометки о том, кто говорит/поет текст:
    if speakers == 'absent': 
        edited = edited
    elif speakers == 'none':
        edited = v2_speakers_formatted(edited)
    else:
        custom_pattern = speakers
        print(f'Паттерн для спикеров: {custom_pattern}')
        edited = custom_speakers_formatted(edited, custom_pattern)

    # облагородим текст в случае, если есть некоторый мусор
    edited = re.sub(re.escape('.:'), ':', edited)
    set_symbols = '///'
    edited = re.sub(fr'\n\n+{set_symbols}', f'\n\n{set_symbols}', edited)
    edited = re.sub(re.escape(')./'), '/', edited)

    # записываем отредактированный текст в файл txt
    # сначала задаем имя для файла в зависимости от типа
    if filename.endswith(('.pdf', '.doc', '.txt')):
        new_name = f'{filename[:-4]}_fin.txt'
    elif filename.endswith('.docx'):
        new_name = f'{filename[:-5]}_fin.txt'
    else:
        new_name = f'{filename[:-5]}_fin.txt' # мало ли, что бывает, но есл что-то не так - лучше отрезать побольше

    # сохранение отредактированного текста на компьютер
    with open(f'C:/Users/User/Desktop/КЛ/либретто/Берем/процесс редактирования/{new_name}', 'w', encoding = 'utf-8') as file:
        file.write(edited)    

    return edited
    

### Примеры работы общей функции
Некоторые случаи для демонстрации работы кода и получаемого результата. Приводить все тексты нет смысла и возможности (часть из них была отформатирована в ходе работы над созданием функций)


In [ ]:
v2_complete_formatting_woptions('Белый. Петербург. Полное либретто.pdf', set_info= 'none', 
                                speakers= r'[\n|\x0c]+?[А-ЯЁ][а-яё-]+?\s?[а-яёI]{1,2}?\s?[а-яёI]{1,2}?\s?[А-ЯЁа-яё-]*?\:[\s\n]*?', 
                                to_remove= [r'\n-+?\n', r'\n\n\n\n\n+', r'\u200b']) 


In [ ]:
v2_complete_formatting_woptions('bal_vampirov_spb.txt', set_info = r'\(.*?\)',
                                speakers = r'\n+[А-ЯЁ]+?[\:\.][\s\n]+?',
                                to_remove='none')

In [ ]:
v2_complete_formatting_woptions('PrimeTime.pdf',
                                set_info=r'\(.{10,300}\)',
                                speakers=r'\n+[А-ЯЁ\n ]{3, 30}',
                                to_remove= [r'#', r'[А-ЯЁ]{1,2}\:', r'\(\d{1,3}\)', r'\d{1,3} (?:такт)? не поём!', r'\d{1,3} --[\s\n]\d{1,3}', r'\d [ХОР|Хор|хор]\:?',
                                            r'\d{1,3} такт (фоном, на[\s\n]диалоге)', r'\(\d{1,3} такт\)', r'[УЖ]\d{1,3}',
                                             r'[А-ЯA-Z]&[А-ЯA-Z]\:', r'Solo [А-ЯA-Z]\s?&\s?[А-ЯA-Z]\:',
                                            r'Solo [А-ЯA-Z]\s?&\s?[А-ЯA-Z] и Solo [А-ЯA-Z]\s?&\s?[А-ЯA-Z]\:',
                                            r'[А-ЯA-Z]&[А-ЯA-Z] и [А-ЯA-Z]&[А-ЯA-Z]\:',
                                            r'\n+МОСКОВСКИЙ ТЕАТР МЮЗИКЛА \nПраймТайм \nv.05.03.2020',
                                            r'\n+\d{1,3}\s?\n+', r'', r'\x0c'
                                            ]
                                )